<a href="https://colab.research.google.com/github/rk1973unsl/notebooks/blob/main/sinusoidal_interactivo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Definiciones de código
ejecutar las proximas celdas antes que las próximas secciones

In [ ]:
#@title Importar Bilioteca de Ploteo
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#@title Importar Biblioteca de Audio
from IPython.display import Audio
from scipy.io import wavfile
from io import BytesIO

In [ ]:
#@title Importar Biblioteca Interact
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
#@title Definir funciones auxiliares
def normalizar(x):
  '''
    reescala los valores de entrada de manera que los valores
    de salida esten entre 0 y 1
  '''
  xn=x-min(x)
  xn=xn/max(xn)
  return xn


def normalizarb(x):
  '''
     reescala los valores de entrada de manera que los
     valores de salida esten entre -1 y 1
  '''
  xn=2*normalizar(x)-1
  return xn

def graft(x,fs,xlim):
    '''
       grafica en el dominio del tiempo
       primer argumento, arreglo de valores de la funcion en el tiempo
       tercer argumento, periodo a muestrear [t_inicial, t_final]
       segundo argumento, frecuencia de muestreo
    '''
    tmax = len(x)/fs
    t = np.arange(0, tmax, step=1. / fs)
    plt.plot(t, x)
    plt.xlim(xlim)
    plt.xlabel("Tiempo (s)")
    plt.ylabel("Amplitud")


def graff1l(x,fs):
    '''
       grafica en el dominio de la frecuencia, espectro de un lado
       primer argumento, arreglo de valores de la funcion en el tiempo
       segundo argumento, frecuencia de muestreo
       entrega componentes en frecuencia (nros. complejos)
    '''
    N = len(x)
    X=np.fft.fft(x)/N
    freqs = np.arange(0, fs / 2, step=fs / N)
    plt.plot(freqs[:(N // 2)], 2*np.abs(X[:(N // 2)]))
    plt.xlabel("Frequencia (Hz)")
    plt.ylabel("Amplitud")
    return X

def graff2l(x,fs):
    '''
       grafica en el dominio de la frecuencia, espectro de dos lados
       primer argumento, arreglo de valores de la funcion en el tiempo
       segundo argumento, frecuencia de muestreo
       entrega componentes en frecuencia (nros. complejos)
    '''
    N = len(x)
    X = np.fft.fft(x)/N
    # Plot the positive frequencies.
    freqsp = np.arange(0, fs / 2, step=fs / N)
    plt.plot(freqsp, np.abs(X[:(N // 2)]))
    # Plot the negative frequencies.
    freqsn = np.arange(-fs / 2, 0, step=fs / N)
    plt.plot(freqsn, np.abs(X[(N // 2):]))
    # Now we can label the x-axis.
    plt.xlabel("Frequencia (Hz)")
    plt.ylabel("Amplitud")
    return X

def graff1lp(x,fs):
    '''
       grafica en el dominio de la frecuencia, espectro de un lado de potencias
       primer argumento, arreglo de valores de la funcion en el tiempo
       segundo argumento, frecuencia de muestreo
       entrega componentes en frecuencia (nros. complejos)
    '''
    N = len(x)
    X=np.fft.fft(x)/N
    X_p = np.abs(X) ** 2
    freqs = np.arange(0, fs / 2, step=fs / N)
    plt.plot(freqs[:(N // 2)], (X_p[:(N // 2)]))
    plt.xlabel("Frequencia (Hz)")
    plt.ylabel("Potencia")
    return X
  
def graff2lp(x,fs):
    '''
       grafica en el dominio de la frecuencia, espectro de dos lados de potencias
       primer argumento, arreglo de valores de la funcion en el tiempo
       segundo argumento, frecuencia de muestreo
    '''
    N = len(x)
    X = np.fft.fft(x)/N
    X_p = np.abs(X)**2
    # Plot the positive frequencies.
    freqsp = np.arange(0, fs / 2, step=fs / N)
    plt.plot(freqsp, X_p[:(N // 2)])
    # Plot the negative frequencies.
    freqsn = np.arange(-fs / 2, 0, step=fs / N)
    plt.plot(freqsn, X_p[(N // 2):])
    # Now we can label the x-axis.
    plt.xlabel("Frequencia (Hz)")
    plt.ylabel("Amplitud")
    return X

def espectro(x,fs):
    '''
       grafica en el dominio de la frecuencia, espectro de dos lados de potencias en escala logaritmica
       primer argumento, arreglo de valores de la funcion en el tiempo
       segundo argumento, frecuencia de muestreo
    '''
    N = len(x)
    X = np.fft.fft(x)/N
    X_pl = np.log(np.abs(X)**2)
    # Plot the positive frequencies.
    freqsp = np.arange(0, fs / 2, step=fs / N)
    plt.plot(freqsp, X_pl[:(N // 2)])
    # Plot the negative frequencies.
    freqsn = np.arange(-fs / 2, 0, step=fs / N)
    plt.plot(freqsn, X_pl[(N // 2):])
    # Now we can label the x-axis.
    plt.xlabel("Frequencia (Hz)")
    plt.ylabel("Amplitud (log)")
    return X

# Tutorial de controles interactivos (se puede saltar)

---



## Interacción dinámica
Intentamos ahora modificar interactivamente los parametros de la sinusoidal. Primero analizamos brevemente el uso de la biblioteca Interact.

(la explicacion del uso de los objetos interactivos original esta en ingles acá: https://colab.research.google.com/github/jupyter-widgets/ipywidgets/blob/master/docs/source/examples/Using%20Interact.ipynb )

La función `interact`  (`ipywidgets.interact`) crea automaticamente los controles de interfaz de usuario (UI) para explorar código y datos de manetra interactiva.

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

##  `interact` básico

En el nivel mas básico, `interact` autogenera los controles UI para los argumentos de funciones, y luego llama a la función con estos argumentos cuando manipulamos interactivamente estos los controles.

Para utilizar `interact`, es necesario definir la función que queremos explorar.
Aqui tenemos una funcion sinusoidal

In [ ]:
def sinus(a,f,p,tmax=5,fs=8000):
    '''
    a - amplitud [adimencional]
    f - frecuencia [Hz]
    p - fase [radianes]
    fs- frecuencia de muestreo [Hz]
    '''
    t = np.arange(0, tmax, step=1. / fs)
    x = a*np.sin(2 * np.pi * f * t +p) ################ Aquí se modifica la frecuencia de la señal
    plt.plot(t, x)
    #plt.xlim(0, .01)
    return x, t

Cuando se pasa esta función como el primer argumento de `interact`, junto con un argumento clave de valor entero (`x=10`), se genera un deslizador enlazado al parametro de la función.

In [ ]:
interact(sinus, a=10,f=440, p=0.);

interactive(children=(IntSlider(value=10, description='a', max=30, min=-10), IntSlider(value=440, description=…

Cuando se mueve el control deslizante, se llama la función y su valor resultante se imprime. 

Si se le pasa `True` o `False`, `interact` genera una casilla de verificación:

Si se le pasa una cadena de caracteres,   `interact` genera una caja de texto.

También se puede usar `interact` como decorador (concepto bastante extraño propio de Python, ver https://recursospython.com/guias-y-manuales/decoradores/).
Esto permite definir una función e interactuar con ella al mismo tiempo. En este ejemplo `interact` trabaja con funciones de multiples argumentos.

In [ ]:
@interact(x=True, y=1.0,z='pepe')
def g(x, y, z):
    return (x, y, z)

interactive(children=(Checkbox(value=True, description='x'), FloatSlider(value=1.0, description='y', max=3.0, …

## Fijando el valor de argumentos con `fixed`

Hay ocasiones en las que se requiere explorar una función `interact`, pero manteniendo uno o mas de sus argumentos en valores especificos. esto se puede lograr pasando valores con la funcion `fixed()`

In [ ]:
interact(sinus, a=10,f=fixed(440), p=fixed(0.));

interactive(children=(IntSlider(value=10, description='a', max=30, min=-10), IntSlider(value=5, description='t…

En este caso solo se genera un control deslizante para el argumento **a**, ya que **f** y **p** estan fijos. Esto sirve para manipular algunas variables y dejar otras fijas sin tener que declarar funciones nuevas. Se define una funcion con multiples argumentos, pero se interactua solo con algunos de estos argumentos. aparecen los deslizadores de los argumentos no declarados pero implicitos **tmax** y **fs**, si quisieramos ocultarlos deberiamos declararlos explicitamente fixed()


## Widget abreviados

Cuando se le pasa un argumento entero con un valor de `10` (`x=10`) a `interact`, se genera un control deslizante de valor entero en un rango de `[-10,+3*10]`. en este caso `10` es una *abreviatura* del widget deslizante real:

```python
IntSlider(min=-10, max=30, step=1, value=10)
```

De hecho, se puede obtener el mismo resultado si se pasa  `IntSlider` y sus detalles como el argumento clave para `x`:

La siguiente tabla resume los diferentes tipos de argumentos y como se mapean a controles interactivos:


|Argumentos clave|Widget|
|-------|:--------|
|`True` o `False`|Checkbox|
|text|`'Hola Mundo'`|
|`valor` o `(min,max)` o `(min,max,step)` enteros|IntSlider|
|`valor` o `(min,max)` o `(min,max,step)` flotantes|FloatSlider|
|`['orange','apple']` o `[('one', 1), ('two', 2)]|Menú desplegable|


Nótese que se usa un menu desplegable si se pasa un diccionario o una lista de tuples (elección discreta de valores significativos) o un deslizador si se le entrega una tupla (significando un rango).

Ya se vio antes como trabajan la caja de texto y la caja de verificación.  Aquí se dan más detalles sobre las diferentes abreviaturas de los deslizadores y desplegables.

Si se pasa un par (2-tuple) de números enteros `(min, max)`, se produce un deslizador con valor de número entero con esos valores mínimos y máximos (inclusive). En este caso, se utiliza el tamaño de paso por defecto de `1`.

Si se pasa una 3-tupla de números enteros `(min,max,paso)`, el tamaño del paso también se puede establecer.

In [ ]:
interact(sinus, a=(0,2.),f=(100,2000,100), p=fixed(0.),tmax=fixed(.01));

interactive(children=(FloatSlider(value=1.0, description='a', max=2.0), IntSlider(value=1000, description='f',…

Se produce un deslizador con valores pflotantes *cualquiera* de los elementos de las tuplas son flotantes. Aquí el mínimo es "0.0", el máximo es "10.0" y el tamaño del paso es "0.1" (por defecto).

Tanto para los deslizadores de valor entero como de valor flotante, se puede elegir el valor inicial del widget pasando un argumento de palabra clave por defecto a la función Python subyacente. Aquí se establece el valor inicial de un deslizador de valor flotante en "5.5".

Los menús desplegables se construyen pasando una lista de cadenas. En este caso, las cadenas se utilizan como nombres en el menú desplegable UI y se pasan a la función Python subyacente.

Si se necesita un menú desplegable que pase valores relacionados con cadenas, a la función Python, puede pasar una lista de pares `('label', valor)`. Los primeros elementos son los nombres en el menú desplegable y los segundos elementos son los valores que son los argumentos pasados a la función Python subyacente.

Finalmente, si se necesita un control más granular que el que ofrece la abreviatura, se puede pasar una instancia de "ValueWidget" como argumento.

Un "ValueWidget" es un widget que pretende controlar un único valor. La mayoría de los widgets [empaquetados con ipywidgets](Widget%20List) heredan de `ValueWidget`. Para más información, vea [esta sección](Widget%20Custom.ipynb#DOMWidget,-ValueWidget-yWidget) sobre los tipos de widgets.

## función `interactive`

Además de `interact`, IPython proporciona otra función, `interactive`, que es útil cuando se quiere reutilizar los widgets que se producen o acceder a los datos que están vinculados a los controles de la interfaz de usuario.

Tenga en cuenta que a diferencia de `interact`, el valor de retorno de la función no se mostrará automáticamente, pero puede mostrar un valor dentro de la función con `IPython.display.display`

Aquí se define una función f() que muestra la suma de sus dos argumentos y devuelve la suma. La línea `display` se puede omitir si no se quiere mostrar el resultado de la función.



In [ ]:
from IPython.display import display
def f(a, b):
    display(a + b)
    return a+b

Hasta aquí, los controles de la interfaz de usuario funcionan como si se hubiera usado `interact`. Se pueden manipular interactivamente y la función será llamada. Sin embargo, la instancia del widget devuelta por `interactive` también te da acceso a los argumentos de la palabra clave actual y devuelve el valor de la función Python subyacente. Es decir que se puede acceder a los valores de los deslizadores mediante código.

Aquí están los argumentos actuales de las palabras clave. Si se vuelve a ejecutar esta celda luego de modificar los controles, los valores se actualizarán. 

## Desabilitando la actualización contante

Cuando se interactúa con funciones lentas, la retroalimentación en tiempo real es una carga en lugar de ser útil.  Véase el siguiente ejemplo:

In [ ]:
def slow_function(i):
    print(int(i),list(x for x in range(int(i)) if 
                str(x)==str(x)[::-1] and 
                str(x**2)==str(x**2)[::-1]))
    return

In [ ]:
%%time
slow_function(1e6)

1000000 [0, 1, 2, 3, 11, 22, 101, 111, 121, 202, 212, 1001, 1111, 2002, 10001, 10101, 10201, 11011, 11111, 11211, 20002, 20102, 100001, 101101, 110011, 111111, 200002]
CPU times: user 567 ms, sys: 988 µs, total: 568 ms
Wall time: 580 ms


Nótese que la salida se actualiza mientras se mueve el mouse en el deslizador. esto no es util para funciones lentas debido al retardo de ejecución:

In [ ]:
from ipywidgets import FloatSlider
interact(slow_function,i=FloatSlider(min=1e5, max=1e7, step=1e5));

interactive(children=(FloatSlider(value=100000.0, description='i', max=10000000.0, min=100000.0, step=100000.0…

Hay dos formas de mitigar esto, o se ejecuta bajo demanda,  o se restringe la ejecución solo cuando se suelta el control (evento mouse-up).

### Función `interact_manual`

La función `interact_manual` provee una variante de la interacción que permite restringir la ejecución de manera de realizarla bajo demanda. Se agrega un botón que permite disparar un evento de ejecución.

In [ ]:
interact_manual(sinus, a=(0,2.),f=(100,2000,100), p=fixed(0.),tmax=fixed(.01));

interactive(children=(FloatSlider(value=1.0, description='a', max=2.0), IntSlider(value=1000, description='f',…

Esto también se puede realizar con `interactive` utilizando un `dict` como segundo argumeto, tal como se muestra a continuación:

In [ ]:
slow = interactive(slow_function, {'manual': True}, i=widgets.FloatSlider(min=1e4, max=1e6, step=1e4))
slow

interactive(children=(FloatSlider(value=10000.0, description='i', max=1000000.0, min=10000.0, step=10000.0), B…

### `continuous_update`

Si se utilizan controles deslizantes, se puede poner el argumento de entrada `continuous_update` en `False`.

`continuous_update` is a un argumento de entrada de los controles deslizantes que restringen la ejecución solo a los eventos de liberación de mouse (cuando se suelta el control deslizante).

In [ ]:
interact(slow_function,i=FloatSlider(min=1e5, max=1e7, step=1e5, continuous_update=False));

interactive(children=(FloatSlider(value=100000.0, continuous_update=False, description='i', max=10000000.0, mi…

## Mas control sobre la intefaz de usuario: `interactive_output`

`interactive_output` provee femas felxibilidad: se piede controlar como se distribuyen los elementros de control de interfaz de usuario.

A diferencia de `interact`, `interactive`, y `interact_manual`, `interactive_output` no genera una interfaz de usuario para los objetos. Esto es muy potente ya que permite crear un pbjeto, ponerlo en una caja, pasar el objeto a `interactive_output`, y mantener el control sobre el objeto y su disposición en pantalla. el siguiente código muestra como poner controles deslizantes lado a lado:

In [ ]:
a = widgets.IntSlider()
b = widgets.IntSlider()
c = widgets.IntSlider()
ui = widgets.HBox([a, b, c])
def f(a, b, c):
    print((a, b, c))

out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

display(ui, out)

Output()

## Argumentos que son mutuamente dependientes

Los argumentos que son dependientes entre si se pueden expresar manualmente usando `observe`. 

En el siguiente ejemplo, una variable se utiliza para describir los limites de la otra. Para mas información vea el [cuaderno de ejemplo de eventos de widgets ](./Widget%20Events.ipynb).

In [ ]:
x_widget = FloatSlider(min=0.0, max=10.0, step=0.05)
y_widget = FloatSlider(min=0.5, max=10.0, step=0.05, value=5.0)

def update_x_range(*args):
    x_widget.max = 2.0 * y_widget.value
y_widget.observe(update_x_range, 'value')

def printer(x, y):
    print(x, y)
interact(printer,x=x_widget, y=y_widget);

interactive(children=(FloatSlider(value=0.0, description='x', max=10.0, step=0.05), FloatSlider(value=5.0, des…

## Parpadeos y saltos en la salida

En ocasiones, se puede notar que la salida interactiva parpadea y salta, causando que la posición de desplazamiento del cuaderno cambie a medida que la salida se actualiza. El control interactivo tiene una disposición, por lo que podemos ajustar su altura a un valor apropiado (elegido manualmente) para que no cambie de tamaño a medida que se actualiza.


In [ ]:
%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

def f(m, b):
    plt.figure(2)
    x = np.linspace(-10, 10, num=1000)
    plt.plot(x, m * x + b)
    plt.ylim(-5, 5)
    plt.show()

interactive_plot = interactive(f, m=(-2.0, 2.0), b=(-3, 3, 0.5))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=0.0, description='m', max=2.0, min=-2.0), FloatSlider(value=0.0, descr…

# **Actividad:** Generar una sinusoidal interactiva

Generar un gráfico de una señal sinusoidal en el dominio del tiempo y de la frecuencia, a la que se le pueda modificar en forma interactiva su amplitud $A$, frecuencia $f$ y fase $\phi$

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
# %matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

def sinusoide(fs, A, f, phi):
    #definición de la sinusoide
    tmax = 1
    t = np.arange(0, tmax, step=1. / fs)
    s = A*np.sin(2 * np.pi * f * t + phi)
    #graficado
    plt.figure(2,figsize=(12,4), dpi=100)
    plt.subplot(1,2,1)
    graft(s,fs,[0,.1])
    plt.subplot(1,2,2)
    graff1l(s,fs)
    plt.show()

interactive_plot = interactive(sinusoide, fs=[500,1000,2000,4000], A=(-2.0, 2.0), f=(50, 2000, 50), phi=(-np.pi,np.pi))
output = interactive_plot.children[-1]
output.layout.widt = '1600px'
interactive_plot

interactive(children=(Dropdown(description='fs', options=(500, 1000, 2000, 4000), value=500), FloatSlider(valu…